## Alignment notebook

In this notebook I will align data from different subject to the fMRI data of subj01 using three different methods: anatomical alignment, a ridge regression and the hyperaligment using full and randomized svd

- [ ] Subj01 are the target data.
- [ ] Using the "test data" as alignment data
- [ ] Remove from the test set the comparison set
- [ ] Clusterize the latent space of CLIP Text Embeddings
- [ ] Use 10%, 25%, 50% and 100% of the test dataset for alignment sampling uniformly from the clusters
- [ ] Perform all alignment fro subj02, subj05, subj07
- [ ] Test reconstructions

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
# from nltools.mask import create_sphere, expand_mask
# from nltools.data import Brain_Data, Adjacency
# from nltools.stats import align
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import warnings
# from mvpa2.suite import 
import numpy as np
from sklearn.linear_model import RidgeCV
import numpy as np
import torch
from os.path import join as opj
from transformers import AutoProcessor, CLIPTextModel
import tqdm
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
# from hyperalign import align
%matplotlib inline
from os import makedirs

In [2]:
source_idx=5
target_idx=7

In [3]:
source_test_data=f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/MNI/nsd_test_fmriavg_nsdgeneral_sub{source_idx}.npy"
source_train_data=f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/MNI/nsd_train_fmriavg_nsdgeneral_sub{source_idx}.npy"
source_test_imgs=f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/MNI/nsd_test_stim_sub{source_idx}.npy"

target_test_data=f"/home/matteo/data/NSD/processed_roi/subj0{target_idx}/MNI/nsd_test_fmriavg_nsdgeneral_sub{target_idx}.npy"


In [4]:
os.makedirs(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}",exist_ok=True)

In [5]:
source_imgs=np.load(source_test_imgs)

In [6]:
source_test=np.load(source_test_data)
source_train=np.load(source_train_data)


target_test=np.load(target_test_data)

## Remove useful indices 
indices=[25,31,68,121,126,318,384,492,531,606,702,860]

indices2=[70,116,165,261,278,363,451,774]
indices3=[41,205,230,411,428,446,502,777]
extra=[95,905]
comparison_indices=indices+indices2+indices3+extra

align_indices=[i for i in  range(len(source_test)) if i not in comparison_indices]

comparison_source=source_test[comparison_indices]

## keep only align indices
source_test=source_test[align_indices]
target_test=target_test[align_indices]

In [7]:
source_imgs.shape

(982, 425, 425, 3)

In [8]:
np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_test_NOTALIGNED_fmriavg_nsdgeneral_sub{source_idx}.npy",source_test)


In [9]:
np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_comparison_NOTALIGNED_fmriavg_nsdgeneral_sub{source_idx}.npy",comparison_source)


In [10]:
np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_test_stim_sub{source_idx}.npy",source_imgs[align_indices])

In [11]:
comparison_imgs=source_imgs[comparison_indices]

## Load captions and perform clustering

In [12]:
base_path="/home/matteo/brain-diffuser/data"

processed_data=opj(base_path,"processed_data",f"subj0{target_idx}")
captions_test_data=opj(processed_data, f"nsd_test_cap_sub{target_idx}.npy")
test_captions=np.load(captions_test_data,allow_pickle=True)
test_captions=[i[0] for i in test_captions]
test_captions=list(np.array(test_captions)[align_indices])

## Clustering

In [13]:
device="cuda:0"
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.layer_norm1.weight', 'vision_model.encoder.layers.1.self_attn.out_proj.bias', 'vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_model.encoder.layers.1.layer_norm2.weight', 'vision_model.encoder.layers.2.self_attn.out_proj.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.bias', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.10.mlp.fc1.bias', 'vision_model.encoder.layers.5.self_attn.out_proj.bias', 'vision_model.embeddings.class_embedding', 'vision_model.encoder.layers.1.layer_norm2.bias', 'vision_model.encoder.layers.6.self_attn.k_proj.weight', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.9.self_attn.q_proj.bias', 'vision_model.encoder.layers.9.layer_norm1.bias', 'vision_model.encoder.layers.10.self_attn.v_proj.bias', 'vision_model.encoder.layer

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [14]:
mini_batch=64
test_clip_txt_cls=[]
for i in tqdm.tqdm(range(int(np.ceil(len(test_captions)/mini_batch)))):
    with torch.no_grad():
        inputs=processor(text=test_captions[mini_batch*i:mini_batch*(i+1)],padding=True,return_tensors="pt").to(device)
        outputs=model(**inputs)
        test_clip_txt_cls.append(outputs.pooler_output.cpu())
test_clip_txt_cls=torch.cat(test_clip_txt_cls,0)

100%|███████████████████████████████████████████| 15/15 [00:01<00:00, 14.28it/s]


In [15]:
kmeans=KMeans(n_clusters=6).fit(test_clip_txt_cls.numpy())
clusters=kmeans.predict(test_clip_txt_cls.numpy())

/home/matteo/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Sample indices in a stratified way for different splits

In [16]:
indices_10_percent,_=train_test_split(np.arange(len(target_test)),train_size=0.1,random_state=42,stratify=clusters)
indices_25_percent,_=train_test_split(np.arange(len(target_test)),train_size=0.25,random_state=42,stratify=clusters)
indices_50_percent,_=train_test_split(np.arange(len(target_test)),train_size=0.5,random_state=42,stratify=clusters)
indices_100_percent=np.arange(len(target_test))

percent_indices={"10":indices_10_percent,"25":indices_25_percent, "50": indices_50_percent, "100": indices_100_percent}

## Alignment with Ridge Regression

In [17]:
for k,v in tqdm.tqdm(percent_indices.items()):
    aligner=RidgeCV(alphas=[1e2,1e3,1e4,5e4], fit_intercept=True)
    aligner.fit(source_test[v],target_test[v])

    aligned_source_test=aligner.predict(source_test)
    aligned_source_train=aligner.predict(source_train)
    aligned_source_comparison=aligner.predict(comparison_source)
    
    #adjust values
    final_mean=target_test.mean(0)
    final_std=target_test.std(0)
    
    
    aligned_source_test_adj=(aligned_source_test-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_test_adj=final_std*aligned_source_test_adj+final_mean

    aligned_source_train_adj=(aligned_source_train-aligned_source_train.mean(0))/(1e-8+aligned_source_train.std(0))
    aligned_source_train_adj=final_std*aligned_source_train_adj+final_mean

    aligned_source_comparison_adj=(aligned_source_comparison-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_comparison_adj=final_std*aligned_source_comparison_adj+final_mean
    
    os.makedirs(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}",exist_ok=True)

    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_train_fmriavg_nsdgeneral_sub{source_idx}_ridge_fraction-{k}.npy",aligned_source_train_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_test_fmriavg_nsdgeneral_sub{source_idx}_ridge_fraction-{k}.npy",aligned_source_test_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_comparison_fmriavg_nsdgeneral_sub{source_idx}_ridge_fraction-{k}.npy",aligned_source_comparison_adj)




100%|█████████████████████████████████████████████| 4/4 [00:54<00:00, 13.64s/it]


In [18]:
np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_comparison_stim_sub{source_idx}.npy",comparison_imgs)

## Alignment with Hyperalignment

In [19]:
class HyperAlign:
    def __init__(self, device,randomize=False,rank=300):
        self.device = device
        self.R=None
        self.c=None
        self.randomize=randomize
        self.rank=rank

        
    def randomized_svd(self,M, rank=None):
        M=M.double()
        m, n = M.shape

        # Step 1: Randomly generate a matrix G
        G = torch.randn(n, self.rank).to(M.device).double()

        # Step 2: Form a matrix H = MG
        H = M @ G

        # Step 3: Orthonormalize H
        Q, _ = torch.linalg.qr(H)

        # Step 4: Form the matrix T = Q.T @ M
        T = Q.T @ M

        # Step 5: Compute the SVD of T
        U_hat, s, V = torch.linalg.svd(T, full_matrices=False)

        # Step 6: Compute U = QU_hat
        U = Q @ U_hat

        return V.T, s, U


    def orthogonal_procrustes(self, A, B):
        A, B = A.to(self.device), B.to(self.device)

        product = A.T @ B
        if self.randomize:
            U, _, V = self.randomized_svd(product, self.rank)
        else:
            U, _, V = torch.linalg.svd(product)
        R = U @ V.T
        c = torch.trace(B.T @ (A @ R)) / torch.trace(A.T @ A)
        return R, c

    def fit(self,source,target):
        R, c = self.orthogonal_procrustes(source, target)
        self.R=R
        self.c=c
        print("fit completed")
    
    def align(self, source):
        
        assert self.R is not None, "Call fit before with source, target before"
        source = source.to(self.device)
        # R, c = self.orthogonal_procrustes(source, target)

        # Align the source to the target
        source_aligned = self.c * source @ self.R

        return source_aligned.cpu()


## Random SVD HyperAlignment

In [20]:
for k,v in tqdm.tqdm(percent_indices.items()):
    device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    hyper_align = HyperAlign(device,randomize=True,rank=4096)

    source = torch.tensor(source_test[v])
    target = torch.tensor(target_test[v])
    hyper_align.fit(source,target)

    aligned_source_test=hyper_align.align(torch.tensor(source_test)).numpy()
    aligned_source_train=hyper_align.align(torch.tensor(source_train)).numpy()
    aligned_source_comparison=hyper_align.align(torch.tensor(comparison_source)).numpy()
    
    #adjust values
    final_mean=target_test.mean(0)
    final_std=target_test.std(0)
    
    
    aligned_source_test_adj=(aligned_source_test-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_test_adj=final_std*aligned_source_test_adj+final_mean

    aligned_source_train_adj=(aligned_source_train-aligned_source_train.mean(0))/(1e-8+aligned_source_train.std(0))
    aligned_source_train_adj=final_std*aligned_source_train_adj+final_mean

    aligned_source_comparison_adj=(aligned_source_comparison-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_comparison_adj=final_std*aligned_source_comparison_adj+final_mean
    
    os.makedirs(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned",exist_ok=True)

    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_train_fmriavg_nsdgeneral_sub{source_idx}_randomhyper_fraction-{k}.npy",aligned_source_train_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_test_fmriavg_nsdgeneral_sub{source_idx}_randomhyper_fraction-{k}.npy",aligned_source_test_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_comparison_fmriavg_nsdgeneral_sub{source_idx}_randomhyper_fraction-{k}.npy",aligned_source_comparison_adj)




  0%|                                                     | 0/4 [00:00<?, ?it/s]

fit completed


 25%|███████████▎                                 | 1/4 [00:09<00:29,  9.76s/it]

fit completed


 50%|██████████████████████▌                      | 2/4 [00:18<00:18,  9.01s/it]

fit completed


 75%|█████████████████████████████████▊           | 3/4 [00:26<00:08,  8.47s/it]

fit completed


100%|█████████████████████████████████████████████| 4/4 [00:34<00:00,  8.56s/it]


## Full SVD Hyperalignment

In [ ]:
for k,v in tqdm.tqdm(percent_indices.items()):
    device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    hyper_align = HyperAlign(device)

    source = torch.tensor(source_test[v])
    target = torch.tensor(target_test[v])
    hyper_align.fit(source,target)

    aligned_source_test=hyper_align.align(torch.tensor(source_test)).numpy()
    aligned_source_train=hyper_align.align(torch.tensor(source_train)).numpy()
    aligned_source_comparison=hyper_align.align(torch.tensor(comparison_source)).numpy()
    
    #adjust values
    final_mean=target_test.mean(0)
    final_std=target_test.std(0)
    
    
    aligned_source_test_adj=(aligned_source_test-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_test_adj=final_std*aligned_source_test_adj+final_mean

    aligned_source_train_adj=(aligned_source_train-aligned_source_train.mean(0))/(1e-8+aligned_source_train.std(0))
    aligned_source_train_adj=final_std*aligned_source_train_adj+final_mean

    aligned_source_comparison_adj=(aligned_source_comparison-aligned_source_test.mean(0))/(1e-8+aligned_source_test.std(0))
    aligned_source_comparison_adj=final_std*aligned_source_comparison_adj+final_mean
    
    os.makedirs(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned",exist_ok=True)

    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_train_fmriavg_nsdgeneral_sub{source_idx}_hyper_fraction-{k}.npy",aligned_source_train_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_test_fmriavg_nsdgeneral_sub{source_idx}_hyper_fraction-{k}.npy",aligned_source_test_adj)
    np.save(f"/home/matteo/data/NSD/processed_roi/subj0{source_idx}/aligned_to_subj0{target_idx}/nsd_comparison_fmriavg_nsdgeneral_sub{source_idx}_hyper_fraction-{k}.npy",aligned_source_comparison_adj)




  0%|                                                     | 0/4 [00:00<?, ?it/s]

In [1]:
print("fine")

fine


In [ ]:
import os